<a href="https://colab.research.google.com/github/Uma1709/deeplearning/blob/main/Language_modeling_using_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/deep learning/Times_of_India_Healines_since_jan_2020_score.csv')
df

,index,Unnamed: 0,S_No,URL,Date,Headline,Headline Link,Positive,Negative,Neutral,Compound
0,0,0,0,https://timesofindia.indiatimes.com/2020/1/1/a...,2020-01-01 00:00:00,Shivin Narang injures his hand on the set of h...,http://timesofindia.indiatimes.com//tv/news/hi...,24.2,0.0,75.8,49.39
1,1,1,1,https://timesofindia.indiatimes.com/2020/1/1/a...,2020-01-01 00:00:00,Allergy cases on the rise in Bengaluru,http://timesofindia.indiatimes.com//life-style...,0.0,0.0,100.0,0.00
2,2,2,2,https://timesofindia.indiatimes.com/2020/1/1/a...,2020-01-01 00:00:00,A grand Hanukkah celebration held in the city,http://timesofindia.indiatimes.com//city/benga...,33.3,0.0,66.7,45.88
3,3,3,3,https://timesofindia.indiatimes.com/2020/1/1/a...,2020-01-01 00:00:00,"I respect my competitors, because they bring o...",http://timesofindia.indiatimes.com//entertainm...,42.2,0.0,57.8,80.74
4,4,4,4,https://timesofindia.indiatimes.com/2020/1/1/a...,2020-01-01 00:00:00,Strong New Year resolutions keep young minds m...,http://timesofindia.indiatimes.com//life-style...,51.2,0.0,48.8,74.30
...,...,...,...,...,...,...,...,...,...,...,...
50007,50007,50007,50007,https://timesofindia.indiatimes.com/2020/7/19/...,2020-07-19 00:00:00,Chemical analysis confirms cobra venom in body,/city/thiruvananthapuram/chemical-analysis-con...,0.0,0.0,100.0,0.00
50008,50008,50008,50008,https://timesofindia.indiatimes.com/2020/7/19/...,2020-07-19 00:00:00,Have spent over Rs 400 crore on Covid fight: G...,/city/chennai/have-spent-over-rs-400-crore-on-...,0.0,20.6,79.4,-38.18
50009,50009,50009,50009,https://timesofindia.indiatimes.com/2020/7/19/...,2020-07-19 00:00:00,"After 16-mth wait, work starts on Mukarba Chow...",/city/delhi/after-16-mth-wait-work-starts-on-m...,0.0,0.0,100.0,0.00
50010,50010,50010,50010,https://timesofindia.indiatimes.com/2020/7/19/...,2020-07-19 00:00:00,More autonomy sought for banks,/city/aurangabad/more-autonomy-sought-for-bank...,0.0,0.0,100.0,0.00


In [ ]:
!pip install tensorflow

In [ ]:
headlines = df['Headline'].tolist()

In [ ]:
# Assuming headlines is a list of strings with some non-text values
headlines = [str(headline) for headline in headlines]
headlines = headlines[:20]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(headlines)
total_words = len(tokenizer.word_index) + 1

In [ ]:
len(headlines)

20

In [ ]:
headlines

['Shivin Narang injures his hand on the set of his show',
 'Allergy cases on the rise in Bengaluru',
 'A grand Hanukkah celebration held in the city',
 'I respect my competitors, because they bring out the best in me: Yash',
 'Strong New Year resolutions keep young minds motivated',
 'Bhuvann Ponnannaa gets a surprise birthday pool party',
 'A grand Hanukkah celebration held in the city',
 'Hiphop Adhi sings a lively kuthu song',
 'Music helped me find my purpose in life, says Bindhumalini',
 'Bhuvann Ponnannaa gets a surprise birthday pool party',
 'Bengaluru students wish for peace and a greener planet',
 'I found peace and closure during my second stint inside Bigg Boss house: Chaitra Kottoor',
 'Big sports events today',
 'Goa: Now, pay full tax on purchase of new vehicles; auto dealers demand waiver till march 31',
 'Goa: Now, pay full tax on purchase of new vehicles; auto dealers demand waiver till march 31',
 'Are you suffering from that ‘January 1 morning-after’ heel pain?',
 '

In [ ]:
input_sequences = []
for line in headlines:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

In [ ]:
predictors, label = input_sequences[:, :-1], input_sequences[:, -1]

In [ ]:
label = tf.keras.utils.to_categorical(label, num_classes=total_words)


In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))  # Reduced embedding dimension
model.add(LSTM(150))  # Reduced LSTM units
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
6/6 [==============================] - 3s 46ms/step - loss: 4.9201
Epoch 2/100
6/6 [==============================] - 0s 41ms/step - loss: 4.8980
Epoch 3/100
6/6 [==============================] - 0s 35ms/step - loss: 4.8562
Epoch 4/100
6/6 [==============================] - 0s 42ms/step - loss: 4.7647
Epoch 5/100
6/6 [==============================] - 0s 39ms/step - loss: 4.6767
Epoch 6/100
6/6 [==============================] - 0s 37ms/step - loss: 4.6342
Epoch 7/100
6/6 [==============================] - 0s 36ms/step - loss: 4.5818
Epoch 8/100
6/6 [==============================] - 0s 37ms/step - loss: 4.5155
Epoch 9/100
6/6 [==============================] - 0s 41ms/step - loss: 4.4397
Epoch 10/100
6/6 [==============================] - 0s 41ms/step - loss: 4.3477
Epoch 11/100
6/6 [==============================] - 0s 38ms/step - loss: 4.2494
Epoch 12/100
6/6 [==============================] - 0s 36ms/step - loss: 4.1652
Epoch 13/100
6/6 [==============================]

In [ ]:
import numpy as np

def generate_text(seed_text, next_words, model, max_sequence_length, tokenizer, temperature=1.0):
    generated_text = seed_text
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted_probabilities = model.predict(token_list, verbose=0)[0]

        # Apply temperature to the predicted probabilities
        predicted_probabilities = np.log(predicted_probabilities) / temperature
        predicted_probabilities = np.exp(predicted_probabilities)
        predicted_probabilities /= np.sum(predicted_probabilities)

        # Sample a word based on the adjusted probabilities
        predicted = np.random.choice(len(predicted_probabilities), size=1, p=predicted_probabilities)[0]

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
        generated_text += " " + output_word
    return generated_text

In [ ]:
seed_text = "Music helped me find my purpose in life"
next_words = 20  # Number of words to generate

generated_text = generate_text(seed_text, next_words, model, max_sequence_length, tokenizer)
print(generated_text)

Music helped me find my purpose in life says bindhumalini in bigg yash house chaitra kottoor second motivated sartaj kottoor till ahuja  heel allergy chaitra kottoor kottoor
